In [ ]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Import API key
from config import gkey

In [ ]:
# Import cities file into the cities_pd DataFrame
cities_df = pd.read_csv("/Users/diegoperezo97/Documents/Tecnológico de Monterrey – Data Analytics Boot Camp/world-weather-analysis/Module 6/Wednesday, February 2nd 2022 /6-2-Student_Resources/05-Stu_Google_Complex/Resources/Cities.csv")
cities_df = cities_df.rename(columns={"City": "city", "State": "state"})
cities_df.head()

In [ ]:
# Add columns for lat, lng, airport name, airport address, airport rating:
# Hint: Be sure to specify "" as the initial value for each column.
cities_df["lat"] = ""
cities_df["lng"] = ""
cities_df["airport_name"] = ""
cities_df["airport_address"] = ""
cities_df["airport_rating"] = ""

In [ ]:
# Loop through the cities_pd and get the lat/long for each city
# Hint: `requests.get(target_url).json()`
# Hint: `for index, row in cities_df.iterrows():`
# Hint: `cities_pd.loc`
params = {"key": gkey}

for index, row in cities_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    city = row["city"]
    state = row["state"]
    params["address"] = f"{city},{state}"
    response = requests.get(base_url, params=params).json()
    cities_df.loc[index, "lat"] = response["results"][0]["geometry"]["location"]["lat"]
    cities_df.loc[index, "lng"] = response["results"][0]["geometry"]["location"]["lng"]
    
cities_df.head()

In [ ]:
# Use the lat/lng we recovered to identify airports
# Hint: Code is similar to above block (i.e. you will need to use cities_pd.iterrows() again)
# Hint: Use the keyword search for "international+airport"
# Hint: Use a try-except to handle airports that are missing ratings
params = {"key": gkey, "radius": 50000, "types": "airport", "keyword": "international airport"}

for index, row in cities_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    lat = row["lat"]
    lng = row["lng"]
    params["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()
    try:
        cities_df.loc[index, "airport_name"] = response["results"][0]["name"]
        cities_df.loc[index, "airport_address"] = response["results"][0]["vicinity"]
        cities_df.loc[index, "airport_rating"] = response["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping")

In [ ]:
# Visualize to confirm airport data appears:
cities_df.head()

# Save Data to csv:
cities_df.to_csv("/Users/diegoperezo97/Documents/Tecnológico de Monterrey – Data Analytics Boot Camp/world-weather-analysis/Module 6/Wednesday, February 2nd 2022 /6-2-Student_Resources/05-Stu_Google_Complex/Resources/CitiesComplete.csv")